In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
!mkdir ~/.kaggle

In [5]:
!cp /content/drive/MyDrive/KAGGLE_API_CREDENTIALS/kaggle.json ~/.kaggle/kaggle.json

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d plameneduardo/a-covid-multiclass-dataset-of-ct-scans

100% 404M/405M [00:22<00:00, 17.6MB/s]
100% 405M/405M [00:22<00:00, 19.0MB/s]


In [8]:
!unzip /content/a-covid-multiclass-dataset-of-ct-scans.zip

Archive:  /content/a-covid-multiclass-dataset-of-ct-scans.zip
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132320.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132449.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132516.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132540.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132608.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132636.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132704.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132740.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132814.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132844.png  
  inflating: New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132926.png  
  inflating: New_Data_CoV2/Covid/Pat

In [9]:
import torch
from torch import nn
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

In [10]:
# Define a transform to convert the images to tensors
transform = transforms.Compose([transforms.Resize((224,224)),  # Resize all images to 224x224
                                transforms.ToTensor()])  # Convert the images to tensors

# Load the images from the 'root' directory
dataset = datasets.ImageFolder('New_Data_CoV2', transform=transform)


In [11]:
test_split = 0.33
shuffle_dataset = True
random_seed=42
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))

In [12]:
if shuffle_dataset:
  np.random.seed(42)
  np.random.shuffle(indices)

train_indices, test_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

BATCH_SIZE = 32
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)



In [14]:
from helper_functions import accuracy_fn
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
class CovNetScan(nn.Module):
  def __init__(self, input_shape,output):
    super().__init__()
    self.layers_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=35),
        nn.BatchNorm1d(35),
        nn.ReLU(),
        nn.Linear(in_features=35, out_features=50),
        nn.BatchNorm1d(50),
        nn.ReLU(),
        nn.Linear(in_features=50, out_features=40),
        nn.Linear(in_features=40, out_features=output),
    )
  
  def forward(self, X):
    return self.layers_stack(X)

In [16]:
input_shape = 3 * 224 * 224
num_classes = 3
model_0 = CovNetScan(input_shape = input_shape , output = num_classes).to(device)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.06)



In [25]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report
import copy
best_report = None
epochs = 100
best_acc = 0.0
best_model_weights = copy.deepcopy(model_0.state_dict())

for epoch in range(epochs):
    model_0.train()
    for X,y in train_loader:
        X,y = X.to(device),y.to(device)
        y_pred = model_0(X)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:

      with torch.inference_mode():
          model_0.eval()
          y_true = []
          y_pred = []
          
          for X_test,y_test in tqdm(test_loader):
              X_test,y_test = X_test.to(device),y_test.to(device)
              y_test_pred = model_0(X_test)
              test_loss = loss_fn(y_test_pred,y_test)
              y_true.extend(y_test.cpu().numpy())
              y_pred.extend(y_test_pred.argmax(dim=1).cpu().numpy())
          
          report = classification_report(y_true=y_true, y_pred=y_pred, target_names=dataset.classes, output_dict=True)
          current_acc = report['accuracy']
          if best_acc < current_acc: 
              best_acc = current_acc
              best_model_weights = copy.deepcopy(model_0.state_dict())
              best_report = report

          

print(best_report)



100%|██████████| 43/43 [00:08<00:00,  5.26it/s]


{'Covid': {'precision': 0.8616438356164383, 'recall': 0.874826147426982, 'f1-score': 0.8681849551414769, 'support': 719}, 'Healthy': {'precision': 0.7689075630252101, 'recall': 0.7408906882591093, 'f1-score': 0.7546391752577319, 'support': 247}, 'Others': {'precision': 0.7965686274509803, 'recall': 0.7926829268292683, 'f1-score': 0.7946210268948656, 'support': 410}, 'accuracy': 0.8263081395348837, 'macro avg': {'precision': 0.809040008697543, 'recall': 0.8027999208384532, 'f1-score': 0.8058150524313582, 'support': 1376}, 'weighted avg': {'precision': 0.825606993554032, 'recall': 0.8263081395348837, 'f1-score': 0.8258833430685149, 'support': 1376}}


In [28]:
model_0.load_state_dict(best_model_weights)
torch.save(model_0.state_dict(),'Covid_CT_Scans')